# Collecte des données d'analyse de produits phytopharmaceutiques diffusées par l'API Hub'Eau Qualité des nappes d'eau
*Auteur : Antonio Andrade, ingénieur de données, [Office français de la biodiversité](https://ofb.gouv.fr/)*  
*Juillet 2023*

Ce notebook illustre l'utilisation de [Hub'Eau](https://hubeau.eaufrance.fr) pour télécharger les analyses de produits phytopharmaceutiques (PPP) diffusées par l'[API Qualité des nappes d'eau](https://hubeau.eaufrance.fr/page/api-qualite-nappes) Ces données sont produites ou collectées par le [BRGM](https://www.brgm.fr/fr) Elles sont rassemblées dans [Ades, la banque nationale des données sur les eaux souterraines](https://ades.eaufrance.fr/) et diffusées sous [Licence ouverte Etalab 2.0](https://www.etalab.gouv.fr/wp-content/uploads/2017/04/ETALAB-Licence-Ouverte-v2.0.pdf).

Le traitement ciblera ici les analyses de PPP qualifiées de `correcte` réalisées entre 2021 et 2022. Cela représente un volume de plus de 8,6 millions d'analyses réalisées sur des dizaines de milliers de stations. Pour tenir compte de la limite de 20000 résultats maximum pour une recherche Hub'Eau, nous allons récupérer les données en suivant les étapes suivantes :
1. Récupération de la liste des départements français
2. Récupération de la liste des stations (point d'eau) pour chaque département
3. Consolidation de la liste des stations sur le territoire national
4. Récupération des analyses de PPP par groupe de 10 stations

## Chargement des modules
Commençons par charger les modules python nécessaires aux traitements :

In [1]:
import calendar
import io
import json
import os
import time

import pandas
import requests

print("Chargement OK")

Chargement OK


## Paramétrage général des traitements
Poursuivons avec le paramétrage général de nos traitements :

In [2]:
# Répertoire d'enregistrement des données téléchargées
data_folder_name = "data_20230721"
download_folder_path = os.getcwd() + "\\" + data_folder_name
if not os.path.exists(download_folder_path):
    os.mkdir(download_folder_path)

print("Paramétrage OK")

Paramétrage OK


## Téléchargement des départements
Commençons notre chaîne de traitement avec le téléchargement des données descriptives des stations de mesure. Leur nombre dépasse la limite de 20000 résultats pour une recherche Hub'Eau. Par conséquent, nous allons récupérer les données en itérant sur les départements. Pour récupérer, les données descriptives de ces derniers, nous allons utiliser l'[API Sandre Référentiel](https://www.sandre.eaufrance.fr/api-referentiel#).

Construisons d'abord l'URL de téléchargement des données des départements :

In [3]:
# URL de base des API Sandre
base = "https://api.sandre.eaufrance.fr"
# Point d'accès de l'API Sandre Référentiel
endpoint = "referentiels/v1"
# Opération de téléchargement des données descriptives des communes administratives
operation = "dep"
# Format des données
format = "json"
# Schéma des données
schema = "SANDREv4"
# Construction de l'URL de téléchargement des données
url = f"{base}/{endpoint}/{operation}.{format}?outputSchema={schema}"
url

'https://api.sandre.eaufrance.fr/referentiels/v1/dep.json?outputSchema=SANDREv4'

Téléchargeons maintenant les données :

In [4]:
# Interrogation de l'API
response = requests.get(url)
# Récupération des données dans un dictionnaire
response = response.json()
response

{'REFERENTIELS@attributes': {'xmlns': 'http://xml.sandre.eaufrance.fr/scenario/referentiel/4',
  'xmlns:xsi': 'http://www.w3.org/2001/XMLSchema-instance'},
 'REFERENTIELS': {'Scenario': {'CodeScenario': 'REFERENTIEL',
   'VersionScenario': '4',
   'NomScenario': 'Diffusion de listes de référence par le Sandre',
   'DateCreationFichier': '2023-06-29',
   'DateDebutReference': '2023-01-01',
   'DateFinReference': '2023-12-31',
   'Emetteur': {'CdIntervenant@attributes': {'schemeAgencyID': 'SANDRE'},
    'CdIntervenant': '1470',
    'NomIntervenant': "Service d'Administration Nationale des Données et Référentiels sur l'Eau"}},
  'Referentiel': {'CdReferentiel@attributes': {'schemeAgencyID': 'SANDRE'},
   'CdReferentiel': 'Departement',
   'NomReferentiel': 'Départements administratifs',
   'TypeReferentiel': '2',
   'StReferentiel': 'Validé',
   'DateMajReferentiel': '2023-06-29T14:01:15',
   'NbOccurrences': '101',
   'SourceReferentiel': '10',
   'VersionReferentiel': '2023',
   'Depart

Récupérons dans la réponse de l'API Les données descriptives des départements : 

In [5]:
departements = response["REFERENTIELS"]["Referentiel"]["Departement"]
departements[0]

{'CdDepartement': '01',
 'LbDepartement': 'Ain',
 'StDepartement': 'Validé',
 'Region': {'CdRegion': '84', 'LbRegion': 'Auvergne-Rhône-Alpes'},
 'CommuneChefLieu': {'CdCommune': '01053',
  'LbCommune': 'Bourg-en-Bresse',
  'SourceReferentiel': '10',
  'VersionReferentiel': '2023'}}

## Téléchargement des données Hub'Eau
Pour faciliter les échanges avec Hub'Eau, définissons une fonction de téléchargement de données :

In [6]:
def get_data(url):
    MAX_TRY = 5 # Nombre max de tentatives d'interrogation
    DELAY_BEFORE_REQUEST = 2 # Délai entre deux tentatives d'interrogation
    MAX_RESULTS = 20000 # Nombre maximum de résultat pour une recherche
    
    try:
        request_number = 0
        while request_number < MAX_TRY:
            response = requests.get(url)
            request_number += 1
            if response.status_code not in range(200, 299):
                #print(f"\t{url} : Tentative {request_number} - Statut {response.status_code} ({response.reason})")
                time.sleep(DELAY_BEFORE_REQUEST)
            else:
                if request_number != 1:
                    #print(f"\t{url} : Tentative {request_number} - Statut {response.status_code}")
                    pass
                break

        if request_number == MAX_TRY:
            print(f"\t{url} : Abandon après {request_number} tentatives")
            return None

        response = response.json()

        if "count" not in response \
            or "data" not in response \
            or "first" not in response \
            or "last" not in response \
            or "prev" not in response \
            or "next" not in response:
            print(f"\t{url} : Réponse invalide")
            return None

        if response['count'] == 0 or len(response['data']) == 0:
            #print(f"\t{url} : Aucun résultat")
            return None
        
        if response['count'] > MAX_RESULTS:
            return None

        return response

    except Exception as e:
        print(f"{url} : Erreur \n{e}")

## Téléchargement des données des stations de mesure
Nous sommes prêts pour utiliser l'API Hub'Eau Qualité des nappes pour récupérer et consolider la liste des stations de mesure. Nous limiterons ici la récupération de données au code BSS de la station.

Construisons d'abord l'URL de téléchargement des données des stations :

In [7]:
# URL de base des API Hub'Eau
base = "http://hubeau.eaufrance.fr/api"
# Point d'accès de l'API Hub'Eau Qualité des nappes d'eau
endpoint = "v1/qualite_nappes"
# Opération de téléchargement des données descriptives des stations
operation = "stations"
# Données souhaitées dans la réponse (cf. données disponibles du modèle Station de mesure de la qualité)
fields = [
    "bss_id", # Identifiant d'une station
]
# Paramètres de la recherche
MAX_RESULTS = 20000
parameters = [
    {
        "parameter": "fields",
        "value": ",".join(fields)
    },
    {
        "parameter": "size",
        "value": MAX_RESULTS # Taille maximale d'une page de résultats de recherche
    }, 
]
# Construction de l'URL de téléchargement des données
parameters = "&".join([f"{param['parameter']}={param['value']}" for param in parameters])
url_base = f"{base}/{endpoint}/{operation}?{parameters}"
url_base

'http://hubeau.eaufrance.fr/api/v1/qualite_nappes/stations?fields=bss_id&size=20000'

Téléchargeons les données des stations par département et rassemblons les résultats dans une liste d'identifiants :

In [8]:
stations = []
for departement in departements:
    # Ajout d'un filtre sur les données du département
    code_departement = departement["CdDepartement"]
    url = url_base + f"&num_departement={code_departement}"
    
    # Interrogation Hub'Eau
    response = get_data(url)
    if not response:
        continue

    # Extraction des données dans la réponse de l'API
    stations += response['data']

print(len(stations))
print(stations[0])

79972
{'bss_id': 'BSS001SDAA'}


## Téléchargement des données des analyses chimiques des nappes d'eau
Nous allons enfin utiliser la liste des stations et l'API Hub'Eau Qualité des nappes pour récupérer les données des analyses chimiques recherchées. Pour optimiser les échanges, nous interrogerons l'API par bloc de 10 stations. L'enregistrement des données s'effectuera dasn des fichiers csv de 100000 lignes environ.

Construisons d'abord l'URL de téléchargement des données des analyses :

In [9]:
# URL de base des API Hub'Eau
base = "http://hubeau.eaufrance.fr/api"
# Point d'accès de l'API Hub'Eau Qualité des nappes d'eau
endpoint = "v1/qualite_nappes"
# Opération de téléchargement des données descriptives des analyses chimiques
operation = "analyses"
# Paramètres de la recherche des analyses
MAX_RESULTS = 20000
parameters = [
    {
        "parameter": "date_debut_prelevement",
        "value": "2021-01-01"
    },
    {
        "parameter": "date_fin_prelevement",
        "value": "2022-12-31"
    },
    {
        "parameter": "code_groupe_parametre",
        "value": "95" # Code Sandre du groupe de paramètres Phytosanitaires, https://id.eaufrance.fr/gpr/95
    },
    {
        "parameter": "code_qualification",
        "value": "1" # Code Sandre de la qualification Correcte, https://id.eaufrance.fr/nsa/414#1
    },
    {
        "parameter": "size",
        "value": MAX_RESULTS # Taille maximale d'une page de résultats de recherche
    }, 
]
# Construction de l'URL de téléchargement des données
parameters = "&".join([f"{param['parameter']}={param['value']}" for param in parameters])
url_base = f"{base}/{endpoint}/{operation}?{parameters}"
url_base

'http://hubeau.eaufrance.fr/api/v1/qualite_nappes/analyses?date_debut_prelevement=2021-01-01&date_fin_prelevement=2022-12-31&code_groupe_parametre=95&code_qualification=1&size=20000'

Téléchargeons les données des analyses chimiques par station :

In [10]:
MAX_CSV_SIZE = 100000
STATION_COUNT = 10

analyses = []
index = 0
i = 0
#for station in stations:
for i in range(0, len(stations), STATION_COUNT):
    bss_ids = [station["bss_id"] for station in stations[i:i + STATION_COUNT]]
    bss_ids = ",".join(bss_ids)
    
    i += 1
    if not i % 1000:
        print(f"Traitement de la {i}ème station...")
    
    # Ajout d'un filtre sur les données des stations
    url = url_base + f"&bss_id={bss_ids}"
    #print(url)
    response = get_data(url)
    if not response:
        continue
       
    # Extraction des données dans la réponse de l'API
    analyses += response['data']

    if len(analyses) > MAX_CSV_SIZE:
        # Création d'un DataFrame
        df = pandas.DataFrame(analyses)
        # Enregistrement du DataFrame dans un fichier CSV
        index += 1
        csv_path = os.path.join(download_folder_path, f"ades_analyses_{index}.csv")
        df.to_csv(
            path_or_buf=csv_path,
            sep=';',
            na_rep='',
            float_format=None,
            columns=None,
            header=True,
            index=False,
            index_label=None,
            mode='w',
            encoding=None,
            compression='infer',
            quoting=None,
            quotechar='"',
            line_terminator=None,
            chunksize=None,
            date_format=None,
            doublequote=True,
            escapechar=None,
            decimal='.'
        )
        # Réinitialisation de la liste d'analyses
        analyses = []

if len(analyses):
    # Création d'un DataFrame
    df = pandas.DataFrame(analyses)
    # Enregistrement du DataFrame dans un fichier CSV
    index += 1
    csv_path = os.path.join(download_folder_path, f"ades_analyses_{index}.csv")
    df.to_csv(
        path_or_buf=csv_path,
        sep=';',
        na_rep='',
        float_format=None,
        columns=None,
        header=True,
        index=False,
        index_label=None,
        mode='w',
        encoding=None,
        compression='infer',
        quoting=None,
        quotechar='"',
        line_terminator=None,
        chunksize=None,
        date_format=None,
        doublequote=True,
        escapechar=None,
        decimal='.'
    )
    # Réinitialisation de la liste d'analyses
    analyses = []

df.head()

,bss_id,code_bss,urn_bss,precision_coordonnees,longitude,latitude,altitude,code_insee_actuel,nom_commune_actuel,num_departement,...,code_qualification,nom_qualification,uri_qualification,limite_quantification,limite_detection,seuil_saturation,incertitude_analytique,codes_groupe_parametre,noms_groupe_parametre,uris_groupe_parametre
0,BSS002PMBD,12307X0021/KAOUE3,http://services.ades.eaufrance.fr/pointeau/BSS...,None,45.219572,-12.761568,21.0,97611,Mamoudzou,976,...,1,Correcte,http://id.eaufrance.fr/NSA/414#1,0.03,0.0100,None,25.0,"[191, 61, 41, 95, 199, 130, 127, 203, 73, 132,...",[Avis relatif aux limites de quantification de...,"[http://id.eaufrance.fr/GPR/191, http://id.eau..."
1,BSS002PMBD,12307X0021/KAOUE3,http://services.ades.eaufrance.fr/pointeau/BSS...,None,45.219572,-12.761568,21.0,97611,Mamoudzou,976,...,1,Correcte,http://id.eaufrance.fr/NSA/414#1,0.03,0.0100,None,NaN,"[148, 121, 167, 50, 150, 31, 149, 207, 191, 12...",[Substances pertinentes complémentaires pour l...,"[http://id.eaufrance.fr/GPR/148, http://id.eau..."
2,BSS002PMBD,12307X0021/KAOUE3,http://services.ades.eaufrance.fr/pointeau/BSS...,None,45.219572,-12.761568,21.0,97611,Mamoudzou,976,...,1,Correcte,http://id.eaufrance.fr/NSA/414#1,0.01,0.0030,None,11.5,"[168, 95, 216, 211, 144, 64, 199, 190, 121, 20...",[Liste des micropolluants de l'analyse réguliè...,"[http://id.eaufrance.fr/GPR/168, http://id.eau..."
3,BSS002PMBD,12307X0021/KAOUE3,http://services.ades.eaufrance.fr/pointeau/BSS...,None,45.219572,-12.761568,21.0,97611,Mamoudzou,976,...,1,Correcte,http://id.eaufrance.fr/NSA/414#1,0.02,0.0070,None,15.0,"[50, 192, 73, 191, 96, 199, 200, 190, 203, 126...","[Micropolluants organiques, Avis relatif aux l...","[http://id.eaufrance.fr/GPR/50, http://id.eauf..."
4,BSS002PMBD,12307X0021/KAOUE3,http://services.ades.eaufrance.fr/pointeau/BSS...,None,45.219572,-12.761568,21.0,97611,Mamoudzou,976,...,1,Correcte,http://id.eaufrance.fr/NSA/414#1,0.05,0.0167,None,NaN,"[168, 211, 95, 207, 209, 190, 121, 113, 191, 1...",[Liste des micropolluants de l'analyse réguliè...,"[http://id.eaufrance.fr/GPR/168, http://id.eau..."
